## Предсказание свободной энергии связывания

В этой практике вы реализуете собственную графовую архитектуру для предсказания свободной энергии связывания двух белков, которая будет более точно учитывать их геометрию, но сохранит инвариантность относительно движений в пространстве.

На практике сделали всю подготовительную работу для проведения экспериментов, а также обнаружили, что в случае простой графовой модели лучший результат дал граф, построенный на атомной структуре интерфейса, но лишённый внутримолекулярных связей, т.е. рёбер, соединяющих атомы одной и той же молекулы.

Однако, наша модель была крайне простой, и в своих экспериментах вы можете обнаружить, что другой представление входных данных в сочетании с более сложной архитектурой сработает ещё лучше. В качестве бонусного задания вы сможете провести любые эксперименты с архитектурой и способом представления данных.

#### Подготовка данных (с практики по GNN)

In [1]:
import sys
from pathlib import Path

import pandas as pd
import torch
import torch.nn.functional as F
from torch import Tensor, nn
from torch.optim import Adam
from torch_geometric.loader import DataLoader
from torch_geometric.nn.conv import (
    GATConv,
    GatedGraphConv,
    GCNConv,
    GraphConv,
    MessagePassing,
)
from torch_geometric.nn.pool import global_mean_pool

sys.path.append(str(Path.cwd().parent))
from assets.utils.affinity_dataset import (
    ATOMS_INDICES,
    AffinityDataset,
    AtomicInterfaceGraphBuilder,
    DataItem,
    InterfaceGraph,
    PlotlyVis,
)

In [2]:
dataset_dir = Path("../assets/datasets/binding_affinity/")
pdb_dir = dataset_dir / "pdb"
train_csv = pd.read_csv(dataset_dir / "affinity_train.csv")
record = train_csv.iloc[0]
item = DataItem(
    uid=record["uid"],
    receptor_chains=record["receptor_chains"],
    ligand_chains=record["ligand_chains"],
    dG=record["dG"],
    pdb=pdb_dir / f'{record["uid"]}.pdb',
)
graph_builder = AtomicInterfaceGraphBuilder(
    interface_distance=5.0, radius=5.0, keep_inner_edges=False
)
graph = graph_builder.build_graph(item)
graph

Data(edge_index=[2, 1532], y=-12.91, atoms=[333], residues=[333], coordinates=[333, 3], receptor_mask=[333], distances=[1532], num_nodes=333)

#### Задание 1 (5 баллов). Реализация E(3)-инвариантной графовой сети

В нашей простой модели мы использовали межатомные расстояния, чтобы построить граф, но далее никакую информацию о геометрии интерфейса не использовали.

Тем не менее, точное относительное положение атомов может существенно определять силу и характер физических взаимодействий.

В этом задании вы реализуете архитектуру графовой сети, которая использует межатомные расстояния при создании сообщений, которыми обмениваются вершины графа. Тем самым результат не будет зависеть от положения и ориентации белкового комплекса в пространстве, но будет явным образом зависеть от геометрии атомных контактов.

Благодаря `pytorch-geometric` реализация таких моделей сравнительно простая, но чтобы не возникло впечатления, что фреймворк делает совсем какую-то магию, перед выполнением задания ознакомьтесь с туториалом по реализации message-passing neural networks: https://pytorch-geometric.readthedocs.io/en/stable/tutorial/create_gnn.html

##### Задание 1.1 (2 балла). E(3)-инвариантный слой графовой сети

Наш слой будет обновлять эмбеддинги вершин в соответствии с уравнением

$h_i^{(t+1)} = \sum_{j \in \mathcal{N}(i)} \text{MLP}^{(t)} \left( \text{concat} (h_i^{(t)}, h_j^{(t)}, e_{ij}) \right)$

т.е. сообщение между вершинами $i$ и $j$ будет формироваться перцептроном, который принимает на вход эмбеддинги вершин и эмбеддинг соединяющего их ребра

Всю работу по распространению сообщений сделает метод `propagate`, вам нужно только реализовать метод `message`, который эти сообщения сформирует

In [ ]:
class InvariantLayer(MessagePassing):
    def __init__(
        self, edge_dim: int, node_dim: int, hidden_dim: int, aggr: str = "sum"
    ) -> None:
        super().__init__(aggr)
        self.message_mlp = nn.Sequential(
            nn.Linear(2 * node_dim + edge_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, node_dim),
        )

    def forward(self, h: Tensor, edge_index: Tensor, edge_attr: Tensor) -> Tensor:
        return self.propagate(edge_index, h=h, edge_attr=edge_attr)

    # Ваше решение
    # def message(self, ...) -> ...:
    #     ...

Минимальный тест на работоспособность:

In [ ]:
h = torch.randn(4, 8)
edge_index = torch.tensor(
    [
        [0, 0, 1, 1, 2],
        [1, 3, 2, 3, 3],
    ]
)
edge_attr = torch.randn(5, 6)

assert InvariantLayer(6, 8, 10).forward(h, edge_index, edge_attr).shape == torch.Size(
    [4, 8]
)

##### Задание 1.2 (3 балла). E(3)-инвариантная графовая сеть

Постройте модель на основе реализованного вами слоя, которая принимает на вход `InterfaceGraph` и возвращает предсказанную свободную энергию связывания.

Отличия от модели с практики небольшие:
1. Вместо слоя `GraphConv` в модели должен быть ваш `InvariantLayer`
2. Нужно преобразовать расстояния с помощью модуля `RadialBasisExpansion` и передавать их в каждый `InvariantLayer` вместе с очередными эмбеддингами вершин.
3. Для достижения нужной точности может потребоваться добавить нормализацию, например `nn.LayerNorm`

Модуль `RadialBasisExpansion` преобразует значения межатомных расстояний в вектор со значениями в [0, 1] с помощью набора радиальных базисных функций. Подумайте, почему такой способ обработки количественных признаков может работать лучше?

In [3]:
class RadialBasisExpansion(nn.Module):
    offset: Tensor

    def __init__(
        self,
        start: float = 3.0,
        stop: float = 10.0,
        num_gaussians: int = 32,
    ):
        super().__init__()
        offset = torch.linspace(start, stop, num_gaussians)
        self.coeff = -0.5 / (offset[1] - offset[0]).item() ** 2
        self.register_buffer("offset", offset)

    def forward(self, dist: Tensor) -> Tensor:
        dist = dist.view(-1, 1) - self.offset.view(1, -1)
        return torch.exp(self.coeff * torch.pow(dist, 2))


# пример использования
dist = torch.tensor([0.1, 1.4, 2.2, 3.5, 4.4])
RadialBasisExpansion(num_gaussians=5).forward(dist).round(decimals=3)

tensor([[0.2530, 0.0290, 0.0010, 0.0000, 0.0000],
        [0.6580, 0.1600, 0.0140, 0.0000, 0.0000],
        [0.9010, 0.3460, 0.0490, 0.0030, 0.0000],
        [0.9600, 0.7750, 0.2300, 0.0250, 0.0010],
        [0.7260, 0.9800, 0.4870, 0.0890, 0.0060]])

In [ ]:
class InvariantGNN(nn.Module):
    def __init__(
        self,
        node_vocab_size: int,  # кол-во типов вершин, например атомов
        node_dim: int,  # размерность эмбеддинга вершины
        edge_dim: int,  # размерность эмбеддинга ребра
        n_layers: int,  # кол-во графовых слоёв
        dropout: float = 0.0,  # dropout rate
    ) -> None:
        super().__init__()
        ...

    def forward(self, batch: InterfaceGraph) -> Tensor: ...

Минимальный тест:

In [ ]:
train_dataset = AffinityDataset(
    datadir=pdb_dir,
    subset_csv=dataset_dir / "affinity_train.csv",
    graph_builder=graph_builder,
)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
batch = next(iter(train_loader))
model = InvariantGNN(
    node_vocab_size=len(ATOMS_INDICES) + 1,
    node_dim=32,
    edge_dim=16,
    n_layers=2,
    dropout=0.1,
)
assert model.forward(batch).shape == torch.Size([4, 1])

#### Задание 2 (4 балла). Обучение модели

Обучите реализованную модель, выведите в конце обучения метрики на тестовой выборке (MAE, корреляции Пирсона и Спирмена).

Ваша задача: добиться MAE < 1.55

Используйте `AtomicInterfaceGraphBuilder(interface_distance=5.0, radius=5.0, keep_inner_edges=False)`, эти параметры можно будет изменить в следующем задании.

Но вы можете выбрать любой размер модели и способ регуляризации, а также любой оптимизатор.

In [ ]:
from scipy.stats import pearsonr, spearmanr


@torch.no_grad()
def validate(loader: DataLoader, model: nn.Module) -> tuple[list[float], list[float]]:
    model.eval()
    ys = []
    yhats = []
    loss = 0.0
    for batch in loader:
        yhat = model.forward(batch)
        yhats.extend(yhat.flatten().tolist())
        ys.extend(batch.y.tolist())
        loss += F.mse_loss(yhat.flatten(), batch.y, reduction="sum").item()

    print(f"Loss: {loss / len(ys):.4f}, ", end="")
    print(f"MAE: {(torch.tensor(ys) - torch.tensor(yhats)).abs().mean():.4f}, ", end="")
    print(f"Pearson R: {pearsonr(ys, yhats).statistic:.4f}, ", end="")
    print(f"Spearman R: {spearmanr(ys, yhats).statistic:.4f}")
    model.train()
    return yhats, ys

In [ ]:
graph_builder = AtomicInterfaceGraphBuilder(
    interface_distance=5.0, radius=5.0, keep_inner_edges=False
)
train_dataset = AffinityDataset(
    datadir=pdb_dir,
    subset_csv=dataset_dir / "affinity_train.csv",
    graph_builder=graph_builder,
)
test_dataset = AffinityDataset(
    datadir=pdb_dir,
    subset_csv=dataset_dir / "affinity_test.csv",
    graph_builder=graph_builder,
)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
torch.manual_seed(42)

model = InvariantGNN(
    node_vocab_size=len(ATOMS_INDICES) + 1,
    node_dim=32,
    edge_dim=32,
    n_layers=3,
    dropout=0.5,
)
optim = Adam(model.parameters(), lr=0.001, weight_decay=0.005)

In [ ]:
for i in range(50):
    model.train()
    for batch in train_loader:
        yhat = model.forward(batch)
        loss = F.mse_loss(yhat.flatten(), batch.y)
        loss.backward()
        optim.step()
        optim.zero_grad()

    if (i + 1) % 5 == 0:
        validate(test_loader, model)

#### Задание 3 (необязательное). В погоне за точностью

Используйте любую графовую архитектуру, чтобы добиться MAE < 1.45.

Баллы за задание:
- 3 балла — за MAE < 1.45
- +1 балл за каждые следующие 0.01

Задание с полной свободой творчества, можно менять и архитектуру модели, и использовать любые модули из `pytorch-geometric`, и менять способ представления данных. Вот некоторые идеи, которые можно тестировать:
1. **Модификация модели с практики**: она является достаточно сильным бейзлайном, поэтому может иметь смысл поколдовать над ней: поменять гиперпараметры, функции активации, используемую функцию ошибки (например huber loss или log-cosh)
2. **Использование информации об аминокислотах**: В наших моделях мы кодируем только тип атома, но никак не используем информацию об аминокислотах, к которым эти атомы относятся. Можно к эмбеддингам атомов добавить эмбеддинги аминокислот, индексы которых находятся в атрибуте `residues`.
3. **Модификация реализованной модели**: тут много вариантов, например
   - добавить линейный слой / перцептрон, который будет в каждом графовом слое преобразовывать эмбеддинг рёбер
   - изменить метод `message`, чтобы иначе формировать сообщения
   - изменить метод `update`, чтобы использовать более гибкий метод агрегации сообщений от соседей; например, реализовать механизм внимания, как в `torch_geometric.nn.conv.GATConv` 
4. **Включение внутрибелковых рёбер**: возможно, модели не хватает обмена информацией с соседними вершинами того же белка, но в архитектуре модели сейчас нет ничего, что учитывает тип ребра: внутреннее (между атомами одного белка) и внешнее (между атомами рецептора и лиганда). Можно добавить в представление ребра его тип: как бинарную переменную или как эмбеддинг (`nn.Embedding(2, edge_dim)`). Получить тип ребра можно из тензоров `edge_index` и `receptor_mask`.